In [1]:
import pandas as pd
import numpy as np

In [2]:
num = pd.read_csv("numerical(1).csv")
num.shape

(95412, 315)

In [3]:
cat = pd.read_csv("categorical.csv")

In [4]:
# transforming Domain_B 
mapping = {1: 4, 2: 3, 3: 2, 4: 1}
cat["DOMAIN_B"] = cat["DOMAIN_B"].replace(mapping)
cat["DOMAIN_B"].value_counts() 

DOMAIN_B
3    48356
4    28498
2    16754
1     1804
Name: count, dtype: int64

In [5]:
#Converting DATASRCE into object
cat["DATASRCE"] = cat["DATASRCE"].astype('object')

In [6]:
# Dropping RFA_2R because it has only one unique value
cat.drop("RFA_2R",axis =1,inplace=True)

In [7]:
target = pd.read_csv("target.csv")


# Look critically at the dtypes of numerical and categorical columns and make changes where appropriate

In [ ]:
# numer =[]
# for i in num.columns:
#     unique_values = num[i].nunique()
#     if unique_values <=10: 
#         numer.append(i)
# # this above code check the number of unique values of each 
# #numerical columns and when the unique values are less than or equal to 10 , we will convert it into category        

In [ ]:
# for i in numer:
#     num[i] = num[i].astype('object')

In [ ]:
# num.columns[num.dtypes==object]

In [ ]:
# for column in numerical.columns:
#     unique_values = numerical[column].unique()
#     print(f"Column: {column}, Unique Values: {unique_values}")

In [ ]:
# All features in numerical are integer or float.
# Even they are categorical, but have dtype integer or float, I will not convert dtype, as this will create too much additional features for the future model.

# concating num and cat = X

In [8]:
X = pd.concat([num,cat],axis =1)

In [9]:
y =target["TARGET_B"]

In [10]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 1)

In [11]:
#Split further into train_num and train_cat. Also test_num and test_cat
train_cat = X_train.select_dtypes(include = "object")
train_num = X_train.select_dtypes(exclude = "object")
test_cat = X_test.select_dtypes(include = "object")
test_num = X_test.select_dtypes(exclude = "object")

In [12]:
# Scale the features using standardscaler

from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(train_num)
X_train_scale = transformer.transform(train_num)
X_test_scale = transformer.transform(test_num)

In [13]:
# Encode the categorical features using One-Hot Encoding

# One hot encoding
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(train_cat)
X_train_one_hot = encoder.transform(train_cat).toarray()
X_test_one_hot= encoder.transform(test_cat).toarray()



In [14]:
#re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
X_train_r = np.concatenate((X_train_scale,X_train_one_hot),axis=1)
X_test_r = np.concatenate((X_test_scale,X_test_one_hot),axis=1)

In [15]:
# Fit a logistic regression (classification) model on the training data.
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [16]:
model = LogisticRegression()
model.fit(X_train_r, y_train)
y_pred = model.predict(X_test_r)

In [17]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ",accuracy)


Accuracy:  0.9487501965099827


In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[18105,     0],
       [  978,     0]], dtype=int64)

In [19]:
score = model.score(X_test_r, y_test)
print("Test Accuracy Score", score)


Test Accuracy Score 0.9487501965099827


# Managing imbalance in the dataset

In [20]:
# Check for the imbalance
target['TARGET_B'].value_counts()

TARGET_B
0    90569
1     4843
Name: count, dtype: int64

In [21]:
X_train_r_s = pd.DataFrame(X_train_r)
X_train_r_s

,0,1,2,3,4,5,6,7,8,9,...,345,346,347,348,349,350,351,352,353,354
0,-0.054979,-1.360613,0.508956,0.349417,-0.250339,0.188977,-1.087130,1.281545,-1.450164,-0.875158,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.055998,-0.003631,0.508956,0.718790,-0.357662,-0.208697,-0.651540,-0.045104,0.243243,-0.420006,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.054979,-0.115145,-1.254835,0.718790,0.930209,-0.208697,-0.825776,1.878537,-1.845292,0.035146,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.054979,-2.052540,-0.666905,0.718790,-0.357662,0.586651,0.916584,1.281545,-0.434120,0.717874,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.054979,-0.003631,-1.254835,-1.866826,-0.357662,-0.208697,-0.303068,1.148880,-0.772801,-1.102734,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76324,-0.054979,0.645974,-1.254835,0.718790,-0.357662,0.387814,0.219640,-0.111436,1.202841,-0.192430,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76325,-0.055998,-1.083842,0.508956,-0.389331,0.071629,-0.208697,-0.390186,-0.177769,0.356137,0.490298,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
76326,-0.053960,-0.945457,0.508956,0.718790,-0.357662,-0.208697,1.003702,-0.509431,0.638372,0.035146,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76327,-0.027471,-0.807072,0.508956,-0.758704,-0.357662,-0.009860,-0.390186,-0.774761,0.807712,-0.192430,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [22]:
y_train = y_train.reset_index(drop = True)

In [23]:
data_B = pd.concat([X_train_r_s,y_train],axis =1)
data_B

,0,1,2,3,4,5,6,7,8,9,...,346,347,348,349,350,351,352,353,354,TARGET_B
0,-0.054979,-1.360613,0.508956,0.349417,-0.250339,0.188977,-1.087130,1.281545,-1.450164,-0.875158,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,-0.055998,-0.003631,0.508956,0.718790,-0.357662,-0.208697,-0.651540,-0.045104,0.243243,-0.420006,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,-0.054979,-0.115145,-1.254835,0.718790,0.930209,-0.208697,-0.825776,1.878537,-1.845292,0.035146,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,-0.054979,-2.052540,-0.666905,0.718790,-0.357662,0.586651,0.916584,1.281545,-0.434120,0.717874,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,-0.054979,-0.003631,-1.254835,-1.866826,-0.357662,-0.208697,-0.303068,1.148880,-0.772801,-1.102734,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76324,-0.054979,0.645974,-1.254835,0.718790,-0.357662,0.387814,0.219640,-0.111436,1.202841,-0.192430,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
76325,-0.055998,-1.083842,0.508956,-0.389331,0.071629,-0.208697,-0.390186,-0.177769,0.356137,0.490298,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
76326,-0.053960,-0.945457,0.508956,0.718790,-0.357662,-0.208697,1.003702,-0.509431,0.638372,0.035146,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
76327,-0.027471,-0.807072,0.508956,-0.758704,-0.357662,-0.009860,-0.390186,-0.774761,0.807712,-0.192430,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0


In [24]:
from sklearn.utils import resample

category_0 = data_B[data_B['TARGET_B'] == 0]
category_1 = data_B[data_B['TARGET_B'] == 1]

In [25]:
category_0.shape

(72464, 356)

In [26]:
category_1.shape

(3865, 356)

# Downsampling (undersampling)

In [27]:
category_0_undersampled = resample(category_0,
                                   replace=False,
                                   n_samples = len(category_1))

In [28]:
print(category_0_undersampled.shape)
print(category_1.shape)

(3865, 356)
(3865, 356)


In [29]:
data_downsampled = pd.concat([category_0_undersampled, category_1], axis=0)
data_downsampled['TARGET_B'].value_counts()

TARGET_B
0    3865
1    3865
Name: count, dtype: int64

In [30]:
y_train_d = data_downsampled["TARGET_B"]
data_downsampled.drop("TARGET_B", axis =1,inplace =True)

In [31]:
X_test_r_s = pd.DataFrame(X_test_r)

In [32]:
model1 = LogisticRegression()
model1.fit(data_downsampled, y_train_d)
y_pred1 = model1.predict(X_test_r_s)

C:\Users\Denish\anaconda1\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
# Calculate accuracy using accuracy_score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred1)
print("Accuracy:", accuracy)

Accuracy: 0.5786825970759315


In [34]:
# Get the model's score (performance on training data)
score = model1.score(X_test_r_s, y_test)
print("Model score:", score)

Model score: 0.5786825970759315


In [35]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

precision = precision_score(y_test, y_pred1)

recall = recall_score(y_test, y_pred1)

cm = confusion_matrix(y_test, y_pred1)

print("Precision:", precision)
print("Recall:", recall)
print("Confusion matrix:")
print(cm)

Precision: 0.06791483113069016
Recall: 0.5674846625766872
Confusion matrix:
[[10488  7617]
 [  423   555]]


In [36]:
# new_y_pred1 = pd.Series(y_pred1)
# new_y_pred1

# Upsampling (oversampling)

In [37]:
category_1_oversampled = resample(category_1,
                                  replace=True,
                                  n_samples = len(category_0))

In [38]:
print(category_0.shape)
print(category_1_oversampled.shape)

(72464, 356)
(72464, 356)


In [39]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)
data_upsampled['TARGET_B'].value_counts()

TARGET_B
0    72464
1    72464
Name: count, dtype: int64

In [40]:
y_train_u = data_upsampled["TARGET_B"]
data_upsampled.drop("TARGET_B", axis =1,inplace =True)

In [41]:
model2 = LogisticRegression()
model2.fit(data_upsampled, y_train_u)
y_pred2 = model2.predict(X_test_r)

C:\Users\Denish\anaconda1\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

precision_u = precision_score(y_test, y_pred2)

recall_u = recall_score(y_test, y_pred2)

cm_u = confusion_matrix(y_test, y_pred2)

print("Precision:", precision_u)
print("Recall:", recall_u)
print("Confusion matrix:")
print(cm_u)

Precision: 0.07139044568988524
Recall: 0.5470347648261759
Confusion matrix:
[[11146  6959]
 [  443   535]]


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(data_upsampled, y_train_u)
print(clf.score(data_upsampled, y_train_u))
print(clf.score(X_test_r, y_test))

y_pred = clf.predict(X_test_r)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6204460145727534
0.6140019913011581


TARGET_B
0    18105
1      978
Name: count, dtype: int64

array([[11181,  6924],
       [  442,   536]], dtype=int64)

# Feature Selection